Notebook showing usage of the data detective.

In [1]:
# To import detective from relative folder
import detective.core as detective
import pandas as pd

To load the local db we supply the path to the sqlite `.db` file 

In [2]:
db = detective.HassDatabase('sqlite:///home-assistant_v2.db') # To init without fetching entities fetch_entities=False

Successfully connected to database sqlite:///home-assistant_v2.db
There are 5 entities with data


In [3]:
db.domains

{'automation', 'camera', 'group', 'history_graph', 'sensor'}

In [4]:
db.entities['sensor']

['sensor.merton',
 'sensor.particles_01um',
 'sensor.particles_025um',
 'sensor.particles_10um',
 'sensor.serial_sensor']

Use `fetch_all_sensor_data()` to fetch all your sensor data into a pandas dataframe in memory. Note that by default the number of states returned is limited but this can optionally be over-ridden as shown below. 

In [5]:
df = db.fetch_all_sensor_data(limit=1e9)

The returned Pandas dataframe has 1545 rows of data.


Take a minute to read about the differences between [wide-form and long-form data](https://altair-viz.github.io/user_guide/data.html#long-form-vs-wide-form-data). The Pandas dataframe we have is in long form.

In [6]:
df.head()

,domain,entity_id,state,last_changed,attributes
0,sensor,sensor.particles_10um,154,2018-12-04 09:47:14.801458,"{""unit_of_measurement"": ""particles"", ""friendly..."
1,sensor,sensor.particles_025um,143,2018-12-04 09:47:14.801221,"{""unit_of_measurement"": ""particles"", ""friendly..."
2,sensor,sensor.particles_01um,107,2018-12-04 09:47:14.800889,"{""unit_of_measurement"": ""particles"", ""friendly..."
3,sensor,sensor.serial_sensor,"{ ""a"": 107, ""b"": 143, ""c"": 154 }",2018-12-04 09:47:14.799862,"{""a"": 107, ""b"": 143, ""c"": 154, ""friendly_name""..."
4,sensor,sensor.particles_10um,155,2018-12-04 09:47:13.888892,"{""unit_of_measurement"": ""particles"", ""friendly..."


It is necessary to do some formatting of the data before we can plot it, and detective provides several functions to assist.